In [41]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import json
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from transformers import BertTokenizer, BertModel

# Define label mapping
label_mapping = {
    'Appeal to authority': 0, 'Appeal to fear/prejudice': 1, 'Bandwagon': 2,
    'Black-and-white Fallacy/Dictatorship': 3, 'Causal Oversimplification': 4,
    'Doubt': 5, 'Exaggeration/Minimisation': 6, 'Flag-waving': 7, 
    'Glittering generalities (Virtue)': 8, 'Loaded Language': 9, 
    "Misrepresentation of Someone's Position (Straw Man)": 10, 
    'Name calling/Labeling': 11, 'Obfuscation, Intentional vagueness, Confusion': 12, 
    'Presenting Irrelevant Data (Red Herring)': 13, 'Reductio ad hitlerum': 14, 'Repetition': 15, 
    'Slogans': 16, 'Smears': 17, 'Thought-terminating cliché': 18, 'Whataboutism': 19
}

# Define the MemeDataset class
class MemeDataset(Dataset):
    def __init__(self, annotation_file, tokenizer, label_mapping):
        with open(annotation_file, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item['text']
        inputs = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
        input_ids = inputs['input_ids'].squeeze()
        attention_mask = inputs['attention_mask'].squeeze()

        label_indices = [self.label_mapping[label] for label in item['labels']]
        labels = torch.zeros(len(self.label_mapping))
        labels[label_indices] = 1

        return input_ids, attention_mask, labels

# Define the MemeClassifier model
class MemeClassifier(nn.Module):
    def __init__(self, text_model, num_labels):
        super(MemeClassifier, self).__init__()
        self.text_model = text_model
        self.classifier = nn.Linear(self.text_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
        text_features = outputs.pooler_output
        logits = self.classifier(text_features)
        return logits

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = MemeClassifier(BertModel.from_pretrained('bert-base-uncased'), num_labels=len(label_mapping))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.BCEWithLogitsLoss()

# Training and validation datasets and dataloaders
train_dataset = MemeDataset(
    annotation_file='C:\\Users\\harih\\Downloads\\27\\semeval2024_dev_release\\semeval2024_dev_release\\subtask1\\train_filtered.json',
    tokenizer=tokenizer,
    label_mapping=label_mapping
)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

val_dataset = MemeDataset(
    annotation_file='C:\\Users\\harih\\Downloads\\27\\semeval2024_dev_release\\semeval2024_dev_release\\subtask1\\validation.json',
    tokenizer=tokenizer,
    label_mapping=label_mapping
)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Training loop with validation
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss, train_correct, train_total = 0, 0, 0
    for input_ids, attention_mask, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        with torch.no_grad():
            predictions = torch.sigmoid(logits).round()
            train_correct += (predictions == labels).all(dim=1).sum().item()
            train_total += labels.size(0)

    train_loss = total_loss / len(train_loader)
    train_accuracy = train_correct / train_total
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss}, Train Accuracy: {train_accuracy}")

    # Validation phase
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    all_labels, all_preds = [], []
    with torch.no_grad():
        all_labels = []
        all_preds = []
        val_loss = 0
        val_correct = 0
        val_total = 0

        for input_ids, attention_mask, labels in val_loader:
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            val_loss += loss.item()

            predictions = torch.sigmoid(logits).round()
            val_correct += (predictions == labels).all(dim=1).sum().item()
            val_total += labels.size(0)
            all_labels.append(labels.cpu())
            all_preds.append(predictions.cpu())

        # Converting tensors to lists for easier handling
        all_labels = [label.numpy().tolist() for label in all_labels]
        all_preds = [pred.numpy().tolist() for pred in all_preds]

        # Decode labels from indices to names
        true_labels_names = [[index_to_label[idx] for idx, present in enumerate(label_row) if present == 1]
                             for batch in all_labels for label_row in batch]
        predicted_labels_names = [[index_to_label[idx] for idx, present in enumerate(pred_row) if present == 1]
                                  for batch in all_preds for pred_row in batch]

        # Print true and predicted label names
        print("True Labels:", true_labels_names)
        print("Predicted Labels:", predicted_labels_names)
        
        print("Length of Tru lsbels",len(true_labels_names))
        print("Length of Predicted lsbels",len(predicted_labels_names))

        # Calculate and print validation loss and accuracy
        print(f"Validation Loss: {val_loss / len(val_loader)}")
        print(f"Validation Accuracy: {val_correct / val_total * 100}%")

Epoch 1/5: 100%|██████████| 1434/1434 [07:55<00:00,  3.01it/s]


Epoch 1/5, Train Loss: 0.26281418806692713, Train Accuracy: 0.029463040446304045
True Labels: [['Doubt', 'Slogans'], ['Exaggeration/Minimisation', 'Smears'], ['Thought-terminating cliché'], ['Glittering generalities (Virtue)'], ['Appeal to authority', 'Slogans', 'Thought-terminating cliché'], ['Flag-waving', 'Loaded Language', 'Name calling/Labeling'], [], ['Smears'], ['Flag-waving'], ['Appeal to authority', 'Name calling/Labeling'], ['Appeal to authority', 'Loaded Language'], ['Black-and-white Fallacy/Dictatorship'], ['Smears'], ['Loaded Language', 'Name calling/Labeling', 'Thought-terminating cliché'], ['Appeal to authority', 'Exaggeration/Minimisation', 'Flag-waving', 'Name calling/Labeling'], ['Name calling/Labeling'], ['Smears'], ['Glittering generalities (Virtue)', 'Name calling/Labeling', 'Smears'], ['Doubt', 'Loaded Language', 'Name calling/Labeling'], ['Doubt', 'Loaded Language'], ['Slogans'], [], ['Appeal to authority', 'Name calling/Labeling', 'Repetition'], ['Doubt'], ['Tho

Epoch 2/5: 100%|██████████| 1434/1434 [07:56<00:00,  3.01it/s]


Epoch 2/5, Train Loss: 0.2196137434393791, Train Accuracy: 0.13737796373779637
True Labels: [['Doubt', 'Slogans'], ['Exaggeration/Minimisation', 'Smears'], ['Thought-terminating cliché'], ['Glittering generalities (Virtue)'], ['Appeal to authority', 'Slogans', 'Thought-terminating cliché'], ['Flag-waving', 'Loaded Language', 'Name calling/Labeling'], [], ['Smears'], ['Flag-waving'], ['Appeal to authority', 'Name calling/Labeling'], ['Appeal to authority', 'Loaded Language'], ['Black-and-white Fallacy/Dictatorship'], ['Smears'], ['Loaded Language', 'Name calling/Labeling', 'Thought-terminating cliché'], ['Appeal to authority', 'Exaggeration/Minimisation', 'Flag-waving', 'Name calling/Labeling'], ['Name calling/Labeling'], ['Smears'], ['Glittering generalities (Virtue)', 'Name calling/Labeling', 'Smears'], ['Doubt', 'Loaded Language', 'Name calling/Labeling'], ['Doubt', 'Loaded Language'], ['Slogans'], [], ['Appeal to authority', 'Name calling/Labeling', 'Repetition'], ['Doubt'], ['Thoug

Epoch 3/5: 100%|██████████| 1434/1434 [07:56<00:00,  3.01it/s]


Epoch 3/5, Train Loss: 0.18168151692709414, Train Accuracy: 0.25383542538354253
True Labels: [['Doubt', 'Slogans'], ['Exaggeration/Minimisation', 'Smears'], ['Thought-terminating cliché'], ['Glittering generalities (Virtue)'], ['Appeal to authority', 'Slogans', 'Thought-terminating cliché'], ['Flag-waving', 'Loaded Language', 'Name calling/Labeling'], [], ['Smears'], ['Flag-waving'], ['Appeal to authority', 'Name calling/Labeling'], ['Appeal to authority', 'Loaded Language'], ['Black-and-white Fallacy/Dictatorship'], ['Smears'], ['Loaded Language', 'Name calling/Labeling', 'Thought-terminating cliché'], ['Appeal to authority', 'Exaggeration/Minimisation', 'Flag-waving', 'Name calling/Labeling'], ['Name calling/Labeling'], ['Smears'], ['Glittering generalities (Virtue)', 'Name calling/Labeling', 'Smears'], ['Doubt', 'Loaded Language', 'Name calling/Labeling'], ['Doubt', 'Loaded Language'], ['Slogans'], [], ['Appeal to authority', 'Name calling/Labeling', 'Repetition'], ['Doubt'], ['Thou

Epoch 4/5: 100%|██████████| 1434/1434 [07:57<00:00,  3.01it/s]


Epoch 4/5, Train Loss: 0.1467501341392555, Train Accuracy: 0.38162482566248257
True Labels: [['Doubt', 'Slogans'], ['Exaggeration/Minimisation', 'Smears'], ['Thought-terminating cliché'], ['Glittering generalities (Virtue)'], ['Appeal to authority', 'Slogans', 'Thought-terminating cliché'], ['Flag-waving', 'Loaded Language', 'Name calling/Labeling'], [], ['Smears'], ['Flag-waving'], ['Appeal to authority', 'Name calling/Labeling'], ['Appeal to authority', 'Loaded Language'], ['Black-and-white Fallacy/Dictatorship'], ['Smears'], ['Loaded Language', 'Name calling/Labeling', 'Thought-terminating cliché'], ['Appeal to authority', 'Exaggeration/Minimisation', 'Flag-waving', 'Name calling/Labeling'], ['Name calling/Labeling'], ['Smears'], ['Glittering generalities (Virtue)', 'Name calling/Labeling', 'Smears'], ['Doubt', 'Loaded Language', 'Name calling/Labeling'], ['Doubt', 'Loaded Language'], ['Slogans'], [], ['Appeal to authority', 'Name calling/Labeling', 'Repetition'], ['Doubt'], ['Thoug

Epoch 5/5: 100%|██████████| 1434/1434 [07:56<00:00,  3.01it/s]


Epoch 5/5, Train Loss: 0.11346561607100704, Train Accuracy: 0.4888423988842399
True Labels: [['Doubt', 'Slogans'], ['Exaggeration/Minimisation', 'Smears'], ['Thought-terminating cliché'], ['Glittering generalities (Virtue)'], ['Appeal to authority', 'Slogans', 'Thought-terminating cliché'], ['Flag-waving', 'Loaded Language', 'Name calling/Labeling'], [], ['Smears'], ['Flag-waving'], ['Appeal to authority', 'Name calling/Labeling'], ['Appeal to authority', 'Loaded Language'], ['Black-and-white Fallacy/Dictatorship'], ['Smears'], ['Loaded Language', 'Name calling/Labeling', 'Thought-terminating cliché'], ['Appeal to authority', 'Exaggeration/Minimisation', 'Flag-waving', 'Name calling/Labeling'], ['Name calling/Labeling'], ['Smears'], ['Glittering generalities (Virtue)', 'Name calling/Labeling', 'Smears'], ['Doubt', 'Loaded Language', 'Name calling/Labeling'], ['Doubt', 'Loaded Language'], ['Slogans'], [], ['Appeal to authority', 'Name calling/Labeling', 'Repetition'], ['Doubt'], ['Thoug

In [72]:
correct = 0
for i in range(0,len(true_labels_names)):
    trueLab = true_labels_names[i]
    half = 0
    #print("TrueLab is ",trueLab)
    #print("Predicted is ",predicted_labels_names[i])
    for j in trueLab:
        if j in predicted_labels_names[i] and len(j) > 0:
            half = half + 1
    #print("Half value is ",half)
    
    if half >= len(trueLab)/2:
        correct = correct + 1  
    
    #print("Correct value is ",correct)

               
print("Correct is ",correct)
print("Accuracy is ",correct/len(true_labels_names))
               
        

Correct is  297
Accuracy is  0.594


In [1]:
hierarchy = {

}

In [3]:
persuasion_hierarchy = {
    'Persuasion': ['Ethos', 'Pathos', 'Logos'],
    'Ethos': ['Ad Hominem', 'Bandwagon', 'Appeal to Authority', 'Glittering Generalities', 'Transfer'],
    'Pathos': ['Appeal to Emotion', 'Exaggeration', 'Loaded Language', 'Flag Waving', 'Appeal to Fear', 'Transfer'],
    'Logos': ['Justification', 'Reasoning', 'Repetition', 'Intentional Vagueness'],
    'Ad Hominem': ['Name Calling', 'Doubt', 'Smears', 'Reductio ad Hitlerum', 'Whataboutism'],
    'Justification': ['Bandwagon', 'Appeal to Authority', 'Flag Waving', 'Appeal to Fear', 'Slogans'],
    'Reasoning': ['Distraction', 'Simplification'],
    'Distraction': ['Straw Man', 'Red Herring', 'Whataboutism'],
    'Simplification': ['Causal Oversimplification', 'Black & White Fallacy', 'Thought Terminating Cliche'],
}

In [5]:
# Function to get all ancestors of a technique in the hierarchy
def get_ancestors(technique, hierarchy):
    ancestors = []
    for parent, children in hierarchy.items():
        if technique in children:
            ancestors.append(parent)
            ancestors.extend(get_ancestors(parent, hierarchy))
    return ancestors

In [7]:
from sklearn.metrics import f1_score

# Assuming your labels are in string format as used in the hierarchy
def hierarchical_f1_score(y_true, y_pred, label_mapping, hierarchy):
    """Calculate the hierarchical F1 score considering the class hierarchy.

    Args:
        y_true (list of list of strings): True labels.
        y_pred (list of list of strings): Predicted labels.
        label_mapping (dict): Mapping from label names to indices.
        hierarchy (dict): Hierarchy of labels.

    Returns:
        float: The hierarchical F1 score.
    """
    # Reverse the label mapping to convert indices to names
    index_to_label = {v: k for k, v in label_mapping.items()}

    # Flatten the true and predicted labels to include their ancestors
    y_true_flat = []
    y_pred_flat = []
    for label_set in y_true:
        for label in label_set:
            label_name = index_to_label[label]
            y_true_flat.extend(get_ancestors(label_name, hierarchy) + [label_name])
    for label_set in y_pred:
        for label in label_set:
            label_name = index_to_label[label]
            y_pred_flat.extend(get_ancestors(label_name, hierarchy) + [label_name])

    # Convert labels names to indices for F1 calculation
    y_true_indices = [label_mapping[label] for label in y_true_flat]
    y_pred_indices = [label_mapping[label] for label in y_pred_flat]

    # Calculate F1 score
    f1 = f1_score(y_true_indices, y_pred_indices, average='micro')
    return f1




In [19]:
# Reverse the label mapping to convert indices to names
index_to_label = {v: k for k, v in label_mapping.items()}
